#  ROHF comparison
In this notebook, we will use psi4 to generate some data. Then we will attempt to use an equivalent method to the ```CIS.py``` file in order to generate data for ROHF references. This will serve as a way to verify the code already written and as a catalyst for outline inspiration.

In [2]:
import psi4
import numpy as np

In [3]:
psi4.set_options({"basis":"sto-3g", "reference": "uhf", "scf_type":"pk", "guess_mix":"True"})
allyl = psi4.geometry("""
0 1
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")
E, wfn = psi4.energy("scf", return_wfn = True)
E

-74.94207989866803

We will now copy the code from the ```CIS.py``` file and generate a datafile for ROHF. It can then be compared to the data in the CUHF files that are already generated.

In [4]:
def displayCISHamiltonian(wfn, geom):
    """displays the CIS hamiltonian in MO basis"""
    # getting the orbital energies
    
    epsilon_a, C_a = wfn.epsilon_a().np, wfn.Ca().np
    epsilon_b, C_b = wfn.epsilon_b().np, wfn.Cb().np
    #C_a[abs(C_a) < 1e-8] = 0
    #C_b[abs(C_b) < 1e-8] = 0
    epsilon = np.append(epsilon_a, epsilon_b) # accounts for the fact that the energies might be different
    sortedorder = np.argsort(epsilon)
    epsilon = np.sort(epsilon)
    
    # make the C matrix => it contains all the orbitals, both alpha and beta
    C = np.block([[C_a, np.zeros(C_a.shape)], [np.zeros(C_b.shape), C_b]]) # accounts for the fact that the orbitals might be different (uhf, cuhf)
    C = C[:, sortedorder] # sort the eigenfunctions
    
    basis = wfn.basisset()
    mints = psi4.core.MintsHelper(basis)

    # getting the two electron integrals in correct basis => we need it in MO basis
    tei = mints.ao_eri().np # given in chemists notation

    #change the basis of the tei
    tei_int = np.kron(np.eye(2), tei)
    tei_big = np.kron(np.eye(2), tei_int.T)

    tei_ao = tei_big.transpose(0, 2, 1, 3) - tei_big.transpose(0, 2, 3, 1) # accounts for both coulomb and exchange, switch to physisists notation
    tei_mo = np.einsum("pQRS,pP->PQRS", np.einsum("pqRS,qQ->pQRS", np.einsum("pqrS,rR->pqRS", np.einsum("pqrs,sS->pqrS", tei_ao, C, optimize=True), C, optimize=True), C, optimize=True), C, optimize=True)
    
    occupied = wfn.nalpha() + wfn.nbeta()
    available = mints.nbf()*2
    virtual = available - occupied
    #getting the excitations
    excitations = []
    for orbital in range(occupied): # for every occupied orbital
        for another_orbital in range(occupied, available): # we can make an excitation to every virtual orbital
            excitations.append((orbital, another_orbital))
    # getting the hamiltonian
    dim = occupied*virtual
    H_cis = np.zeros((dim, dim))
    for row, excitation in enumerate(excitations):
        i, a = excitation
        for collumn, another_excitation in enumerate(excitations):
            j, b = another_excitation   
            H_cis[row, collumn] = (epsilon[a] - epsilon[i])*(i == j)*(a == b) + tei_mo[a, j, i, b]
    

    # get the E_0 value in the hamiltonian
    extra_row = np.zeros((H_cis.shape[1],))
    H_cis = np.vstack((extra_row, H_cis))
    extra_collumn = np.zeros((H_cis.shape[0], 1))
    H_cis = np.hstack((extra_collumn, H_cis))
    H_cis[0,0] = E
    return H_cis, excitations
ham, excitations = displayCISHamiltonian(wfn, allyl)

In [5]:
from scipy.linalg import eigh
exits, coefs = eigh(ham)

In [6]:
def GetExitations(exits, coefs, excitations, filepath="NoNameGiven"):
        """Get the excitation energies and the contributions"""
        if filepath == "NoNameGiven":
            raise ValueError("no path specified")
        from pathlib import Path
        Path(f"{filepath}").touch()
        datafile = open(f"{filepath}", "w")
        contrib = coefs[1:, 1:]**2
        energies = exits[1:]
        counterdict = {} # added to check how many times a certain excitation occurs
        datafile.writelines(f"scf energy for ROHF: {E}\n")
        for state, energy in enumerate(energies):
            datafile.writelines(f" {state + 1} : {energy}\n")
            for excitation, contribution in enumerate(contrib[:, state]):
                if contribution*100 > 1:
                    datafile.writelines(f"\t{contribution:.3f} : {excitations[excitation]}\n")
                    if excitations[excitation] not in counterdict:
                        counterdict[excitations[excitation]] = 0
                    counterdict[excitations[excitation]] += 1   
        datafile.close()
GetExitations(exits, coefs, excitations, "/user/gent/440/vsc44013/Bachelor_project/Bachelor_project/definitive_versions/notebooks/testdata.txt")

In [7]:
wfn.Ca().np

array([[ 9.94434590e-01,  2.39158839e-01,  8.52022481e-08,
         9.36832470e-02, -6.85965486e-16,  1.11639947e-01,
        -4.07723715e-09],
       [ 2.40970420e-02, -8.85735530e-01, -3.91637078e-07,
        -4.79585854e-01,  3.13317650e-15, -6.69579108e-01,
        -1.93474903e-08],
       [ 3.16154870e-03, -8.58961576e-02,  1.05509145e-06,
         7.47431435e-01, -4.55815582e-15, -7.38488550e-01,
        -6.93171021e-07],
       [ 1.59828206e-16,  4.28969962e-16,  6.71692460e-16,
         6.52490425e-15,  1.00000000e+00,  8.25202595e-17,
         1.21415680e-16],
       [ 2.54829622e-09, -5.92205754e-08,  6.07285064e-01,
        -1.82279881e-07, -2.82107439e-16, -1.72774498e-07,
         9.19234090e-01],
       [-4.59374206e-03, -1.44039621e-01, -4.52997422e-01,
         3.29472034e-01, -1.89739864e-15,  7.09849047e-01,
         7.32460824e-01],
       [-4.59374345e-03, -1.44039648e-01,  4.52997710e-01,
         3.29470227e-01, -2.33852586e-15,  7.09849890e-01,
        -7.3246063

In [8]:
wfn.Cb().np

array([[ 9.94434591e-01,  2.39158828e-01,  9.39974322e-08,
         9.36833280e-02,  3.58269433e-16,  1.11639897e-01,
         5.65259919e-08],
       [ 2.40970392e-02, -8.85735495e-01, -4.93314056e-07,
        -4.79586211e-01, -1.58363640e-15, -6.69578898e-01,
        -3.93650970e-07],
       [ 3.16154883e-03, -8.58964320e-02, -2.30903890e-07,
         7.47431258e-01,  2.32157166e-15, -7.38488697e-01,
        -2.82884755e-07],
       [ 5.48875042e-18, -7.82535707e-17,  1.31291628e-16,
        -3.14205279e-15,  1.00000000e+00, -1.63281724e-16,
        -5.94086472e-17],
       [ 2.53500719e-09,  2.74325632e-08,  6.07284592e-01,
        -2.77060461e-07, -1.22950324e-16, -8.25596614e-07,
         9.19234402e-01],
       [-4.59374152e-03, -1.44039495e-01, -4.52997576e-01,
         3.29471177e-01,  1.13055742e-15,  7.09849194e-01,
         7.32460996e-01],
       [-4.59374312e-03, -1.44039760e-01,  4.52998308e-01,
         3.29471313e-01,  1.02920604e-15,  7.09849641e-01,
        -7.3245999